# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
type(student_data)
len(student_data)
student_data.count
student_data.shape
student_data.describe()
student_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 0 to 394
Data columns (total 31 columns):
school        395 non-null object
sex           395 non-null object
age           395 non-null int64
address       395 non-null object
famsize       395 non-null object
Pstatus       395 non-null object
Medu          395 non-null int64
Fedu          395 non-null int64
Mjob          395 non-null object
Fjob          395 non-null object
reason        395 non-null object
guardian      395 non-null object
traveltime    395 non-null int64
studytime     395 non-null int64
failures      395 non-null int64
schoolsup     395 non-null object
famsup        395 non-null object
paid          395 non-null object
activities    395 non-null object
nursery       395 non-null object
higher        395 non-null object
internet      395 non-null object
romantic      395 non-null object
famrel        395 non-null int64
freetime      395 non-null int64
goout         395 non-null int64
Dalc          395 no

In [4]:
len(student_data[student_data['passed'] == 'yes'])

265

In [5]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns[:-1]) 
n_passed = len(student_data[student_data['passed'] == 'yes'])
n_failed = len(student_data[student_data['passed'] == 'no'])
grad_rate = (float(n_passed) / float(n_students) ) * 100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(
X_all, y_all, test_size=0.4, random_state=0)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 237 samples
Test set: 158 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [9]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.005


In [10]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.86684073107


In [11]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.777777777778


In [12]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
    
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [13]:
# TODO: Train and predict using two other models
def train_and_predict(clf, X_train, y_train, X_test, y_test):
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    f1_score_test = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {}".format(f1_score_test)
    return f1_score_test
    

In [25]:
def test_model(clf, X_all, y_all): 
    print "Training {}...".format(clf.__class__.__name__)
    training_size_n = [300.] #[100., 200., 300.]
    for num_students in training_size_n: 
        training_size = num_students / float(n_students) 
        test_size = 1. - training_size
        from sklearn import cross_validation
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size = test_size, random_state=0)
        print "Training set: {} samples".format(X_train.shape[0])
        print "Test set: {} samples".format(X_test.shape[0])
        test_score = train_and_predict(clf, X_train, y_train, X_test, y_test)
        return { 'Classifier' : clf.__class__.__name__ , 'test_score':  test_score }

    
scores = []    
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
scores.append(test_model(clf, X_all, y_all))

from sklearn import tree
clf = tree.DecisionTreeClassifier( min_samples_split = 40)
scores.append(test_model(clf, X_all, y_all) )

from sklearn import svm
clf = svm.SVC(kernel = 'rbf')
scores.append(test_model(clf, X_all, y_all))

from sklearn.ensemble import GradientBoostingClassifier 
clf = GradientBoostingClassifier()
scores.append(test_model(clf, X_all, y_all))

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
scores.append(test_model(clf, X_all, y_all))


# import pdb
# pdb.set_trace()
# import matplotlib.pyplot as plt
# import pylab as pl

# #type(X_all)


Training GaussianNB...
Training set: 300 samples
Test set: 95 samples
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.808823529412
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.75
Training DecisionTreeClassifier...
Training set: 300 samples
Test set: 95 samples
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.852607709751
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.791044776119
Training SVC...
Training set: 300 samples
Test set: 95 samples
Training SVC...
Done!
Training time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.869198312236
Predicting labels using SVC...

In [27]:
scores = sorted(scores, key=lambda x: x['test_score']) 

for score in scores:
    print score['Classifier'] + ' ' +  str(score['test_score'])

GaussianNB 0.75
SVC 0.758620689655
GradientBoostingClassifier 0.772727272727
RandomForestClassifier 0.772727272727
DecisionTreeClassifier 0.791044776119


In [15]:
from sklearn import tree
clf = tree.DecisionTreeClassifier( min_samples_split = 40)
clf = test_model(clf, X_all, y_all) 
f_indexes = clf.feature_importances_
#print np.argmax(clf.feature_importances_)
print f_indexes.argsort()[-3:][::-1]
print X_all.columns[32]
print X_all.columns[47]

print X_all.columns[42]

Training DecisionTreeClassifier...
Training set: 300 samples
Test set: 95 samples
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.852607709751
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.791044776119


AttributeError: 'dict' object has no attribute 'feature_importances_'

In [ ]:
 d = {'yes': True, 'no': False}
temp3 = pd.crosstab([X_all.freetime, X_all.failures], y_all.map(d))
temp3.plot(kind='bar', stacked=True, color=['red','blue'], grid=False)


In [ ]:
def graph_f(f1, f2, y): 
    pass_f1 = [f1[ii] for ii in range(len(f1)) if y[ii] =='yes']
    pass_f2 = [f2[ii] for ii in range(len(f2)) if y[ii] =='yes']
    fail_f1 = [f1[ii] for ii in range(len(f1)) if y[ii] =='no']
    fail_f2 = [f2[ii] for ii in range(len(f2)) if y[ii] =='no']
    plt.scatter(fail_f1, fail_f2, color = "r", label="Failed")
    plt.scatter(pass_f1, pass_f2, color = "k", label="Passed")

    plt.legend()
    plt.xlabel("abscences")
    plt.ylabel("failures")
x_b = list(X_all['freetime'])
y_b = list(X_all['failures'])
#X_all['']
print X_all.columns[47]
graph_f(x_b, y_b, y_all)



print X_all.columns[47]

In [ ]:


from sklearn.decomposition import PCA
import pylab as pl

pca = PCA(n_components=2)

pca.fit(X_all[1:100])
X = pca.transform(X_all)

import matplotlib.pyplot as plt
import pylab as pl
y_t = y_all.tolist()
# for ii in range(len(X)): 
#     print y_t[ii]
# import pdb
#pdb.set_trace()

pass_f1 = [X[ii][0] for ii in range(len(X)) if y_all[ii] =='yes']
pass_f2 = [X[ii][1] for ii in range(len(X)) if y_all[ii] =='yes']
fail_f1 = [X[ii][0] for ii in range(len(X)) if y_all[ii] =='no']
fail_f2 = [X[ii][1] for ii in range(len(X)) if y_all[ii] =='no']

# grade_sig = [X[ii][0] for ii in X if y_all[ii]=='no']
# bumpy_sig = [X[ii][1] for ii in X if y_all[ii]=='no']
# grade_bkg = [X[ii][0] for ii in X if y_all[ii]=='yes']
# bumpy_bkg = [X[ii][1] for ii in X if y_all[ii]=='yes']
plt.scatter(pass_f1, pass_f2, color = "b", label="Passed")
plt.scatter(fail_f1, fail_f2, color = "r", label="Failed")
plt.legend()
plt.xlabel("f1")
plt.ylabel("f2")
plt.show()
# print 're'



In [ ]:
pca.get_params()


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [34]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
X_all, y_all, test_size=0.2, random_state=0)

param_grid = {
        'max_depth': (50, 100, 150),
        'min_samples_split': (1, 2, 3),
        'min_samples_leaf': (1, 2, 3)
    }
from sklearn import tree

clf = tree.DecisionTreeClassifier()
t = GridSearchCV(clf, param_grid)
t = t.fit(X_all, y_all)

pred = t.predict(X_all)
print "Best estimator found by grid search:"
print t.best_estimator_


from sklearn import cross_validation
    
from sklearn.metrics import accuracy_score
print accuracy_score(pred, y_all)
print accuracy_score(pred_2,y_test)

Best estimator found by grid search:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=150,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
0.936708860759


NameError: name 'pred_2' is not defined